In [1]:
# installation
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai langchain_community tavily-python graphviz

In [3]:
# Setup All .env Files

import os
from google.colab import userdata


os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "quickstart"
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [7]:
import sqlite3
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

#Set up SQLite Database
def init_db():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_input TEXT NOT NULL,
            bot_response TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

def save_to_db(user_input, bot_response):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO chat_history (user_input, bot_response) VALUES (?, ?)", (user_input, bot_response))
    conn.commit()
    conn.close()

def fetch_history():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM chat_history ORDER BY timestamp DESC LIMIT 10")
    rows = cursor.fetchall()
    conn.close()
    return rows

# New: Function to delete a record by ID
def delete_from_db(conversation_id):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM chat_history WHERE id = ?", (conversation_id,))
    conn.commit()
    conn.close()

# New: Function to update a record by ID
def update_record(conversation_id, new_user_input=None, new_bot_response=None):
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    if new_user_input:
        cursor.execute("UPDATE chat_history SET user_input = ? WHERE id = ?", (new_user_input, conversation_id))
    if new_bot_response:
        cursor.execute("UPDATE chat_history SET bot_response = ? WHERE id = ?", (new_bot_response, conversation_id))
    conn.commit()
    conn.close()

# New: Function to fetch all records
def fetch_all_history():
    conn = sqlite3.connect("chatbot_data.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM chat_history ORDER BY timestamp DESC")
    rows = cursor.fetchall()
    conn.close()
    return rows

# Set up Tavily search tool
tavily_tool = TavilySearchResults(max_results=3)

# Define the conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    temperature=0.7,
    max_tokens=200,
    api_key=gemini_api_key
)


# Initialize tools
tools = [
    Tool(
        name="TavilySearch",
        func=tavily_tool.run,
        description="Useful for answering questions requiring research or external data."
    )
]

# Create the conversational agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# Implement the chatbot function
def chatbot():
    print("˗ˏˋ ★ ˎˊ˗ welcome to the Chatbot... Type 'exit', 'quit', 'q' to end the conversation.\n")
    print("Type 'history' to view the last 10 conversations.\n")
    print("Type 'delete <id>' to delete a conversation by ID.\n")
    print("Type 'update <id> <new_user_input> <new_bot_response>' to update a conversation.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            print("「 ✦ Goodbye ✦ 」...👋")
            break
        elif user_input.lower() in ["data", "history", "h"]:
            history = fetch_history()
            if not history:
                print("No history found.")
            else:
                for idx, (id, user, bot, timestamp) in enumerate(history, 1):
                    print(f"{idx}. {timestamp} - You: {user} | Bot: {bot}")
        elif user_input.lower().startswith("delete"):
            try:
                conversation_id = int(user_input.split()[1])
                delete_from_db(conversation_id)
                print(f"Deleted conversation with ID {conversation_id}.")
            except (IndexError, ValueError):
                print("Invalid format. Use 'delete <id>'.")
        elif user_input.lower().startswith("update"):
            try:
                parts = user_input.split(maxsplit=3)
                conversation_id = int(parts[1])
                new_user_input = parts[2]
                new_bot_response = parts[3]
                update_record(conversation_id, new_user_input, new_bot_response)
                print(f"Updated conversation with ID {conversation_id}.")
            except (IndexError, ValueError):
                print("Invalid format. Use 'update <id> <new_user_input> <new_bot_response>'.")
        else:
            response = agent.run(user_input)
            print(f"Bot: {response}")
            save_to_db(user_input, response)

# Initialize and run the chatbot
if __name__ == "__main__":
    init_db()
    chatbot()

˗ˏˋ ★ ˎˊ˗ welcome to the Chatbot... Type 'exit', 'quit', 'q' to end the conversation.

Type 'history' to view the last 10 conversations.

Type 'delete <id>' to delete a conversation by ID.

Type 'update <id> <new_user_input> <new_bot_response>' to update a conversation.

You: q
「 ✦ Goodbye ✦ 」...👋
